![](../images/studio_g.jpeg)
<br>
Photo: [*Studio G*](http://www.studiogbrooklyn.com/studio/)

# Who Do You Sound Like?
### Notebook 3: Exploratory Data Analysis
#### Adam Zucker
---

## Contents
- 
- 
- 

---
### Section 1
#### Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp

import os
import IPython.display as ipd

import librosa as lib
import librosa.display as libd